#### This is the code for named entity recognition. The final model chosen for the thesis


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/

Mounted at /content/drive
MyDrive


# Ner bert

In [ ]:
!pip install pandas
!pip install nltk
!pip install transformers
!pip install tqdm
!pip install spacy


In [ ]:
import pandas as pd
import nltk
from tqdm import tqdm
from transformers import BertTokenizer, BertForTokenClassification, BertConfig
from transformers import pipeline
import ast
import spacy
from collections import Counter

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Export the DataFrame as a CSV file
df_ner.to_csv('/mnt/drive/MyDrive/Colab Notebooks/ner_predictions_dtm_spacy2_top_topic.csv', index=False)

# Display the DataFrame
print(df_ner.head(20))
# Create a dictionary to store entities by topic
entities_by_topic = {topic: [] for topic in topics_of_interest}

for sentence, topic, entities in predictions:
    entities_by_topic[topic].extend(entities)

# Compute frequency of each entity per topic
summary_statistics = {}

for topic, entities in entities_by_topic.items():
    counter = Counter(entities)
    summary_statistics[topic] = counter.most_common()

# Create a DataFrame to display the summary statistics
summary_data = []

for topic, entity_list in summary_statistics.items():
    for (entity_text, entity_type), count in entity_list:
        summary_data.append([topic, entity_text, entity_type, count])

df_summary = pd.DataFrame(summary_data, columns=['Topic', 'Entity', 'Type', 'Count'])

# Export the DataFrame as a CSV file
df_summary.to_csv('/mnt/drive/MyDrive/Colab Notebooks/ner_spacy2_summary_statistics.csv', index=False)

# Display the DataFrame
print(df_summary.head(20))


                                             Sentence  \
0   Given the power relations in Congress, this is...   
1   They contrast the ineffectual "pork-laden" leg...   
2   In December, the EPA issued an "endangerment" ...   
3   Hence halting the legislation in the Senate is...   
4   The major bills are now being pushed by John K...   
5   It is also based on "cap and trade" in which p...   
6   The problem of relying upon these actors to sa...   
7   The boiler deals give the ruling party a multi...   
8   If Zuma aims, therefore, to persuade the world...   
9   The cuts promised by Sonjica just before Copen...   
10  In reality, cuts promised in the Long-Term Mit...   
11  As the Eskom tariff hearings organised by the ...   
12  In reality, cuts promised in the Long-Term Mit...   
13  As the Eskom tariff hearings organised by the ...   
14  Yet the purpose of the new build programme is ...   
15  Environment and social justice activists are c...   
16  The decision to grant the l

# Now I try to give some custom help to the model.

in partiuclar by creating the ruler...

In [ ]:

# Read the CSV file
path_to_csv = '/content/drive/MyDrive/Colab Notebooks/coal_data_filtered_nodup_cleaned3.csv'
df = pd.read_csv(path_to_csv)

# Load the file with the info on top topic per document
path_to_df_top = '/content/drive/MyDrive/Colab Notebooks/topic_model_3_k20_srl_2.csv'
df_top = pd.read_csv(path_to_df_top)

# Merge the datasets based on id column
df = df.merge(df_top, left_on='Unnamed: 0.1', right_on='Unnamed: 0.1', how='left')

# Function to clean text
def clean_text(text):
    # Replace \' with '
    text = text.replace("\'", "")
    # Replace \" with "
    text = text.replace('\"', "")
    text = text.replace('\"', '')
    text = text.replace('\s', '')
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')


    return text

# Clean Article_Text column
df['Article_Text'] = df['Article_Text'].apply(clean_text)


# Extract Year and Month from the Date column
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

print("Merged DataFrame:")
print(df.head())

# List unique topics in the DataFrame
unique_topics = df['Dominant Topic'].unique()
print("Unique topics in the DataFrame:", unique_topics)

# Filter out the topics of no interest
topics_of_no_interest = ["Mining market dynamics", "Stopwords/False Positives 1", "Stopwords/False Positives 2", "Indian South Africans history", "Coal transport"]
df = df[~df['Dominant Topic'].isin(topics_of_no_interest)]

print(f"Filtered DataFrame has {len(df)} rows after filtering by topics.")

# Extract necessary columns
df_subset = df[['Article_Text', 'Dominant Topic', 'Year', 'Month']]

print(f"Subset DataFrame has {len(df_subset)} rows.")


# Initialize an empty list to store sentences and topics
data = []

# Function to truncate sentences
def truncate_sentence(sentence, max_length=50):
    words = sentence.split()
    if len(words) > max_length:
        sentence = ' '.join(words[:max_length])
    return sentence

for index, row in df_subset.iterrows():
    text = row['Article_Text']
    topic = row['Dominant Topic']
    year = row['Year']
    month = row['Month']
    # Clean text before processing
    cleaned_text = clean_text(text)
    # Split the text into sentences using NLTK's sent_tokenize function
    text_sentences = nltk.sent_tokenize(cleaned_text)
    # Check each sentence for the presence of the word "coal"
    for i, sentence in enumerate(text_sentences):
        # Convert the sentence to lowercase for case-insensitive matching
        sentence_lower = sentence.lower()
        # Check if the sentence contains the word "coal"
        if "coal" in sentence_lower:
            # Collect the sentence before, the sentence with "coal", and the sentence after
            before = text_sentences[i-1] if i > 0 else ""
            after = text_sentences[i+1] if i < len(text_sentences) - 1 else ""
            sentences_to_add = [before, sentence, after]
            for sent in sentences_to_add:
                if sent:  # Only add non-empty sentences
                    truncated_sentence = truncate_sentence(sent)
                    data.append((truncated_sentence, topic, year, month))


print(f"Collected {len(data)} sentences containing the word 'coal' and their surrounding context.")

# Load the pre-trained spaCy model for NER
nlp = spacy.load('en_core_web_sm')

# Create a custom entity ruler to handle specific cases
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = [
    {"label": "ORG", "pattern": "Coal of Africa"},
    {"label": "LOC", "pattern": "Mpumalanga"},
    {"label": "ORG", "pattern": "Just Coal"},
    {"label": "ORG", "pattern": "Cosatu"},
    {"label": "ORG", "pattern": "Just Share"},
    {"label": "ORG", "pattern": "Greenpeace Africa"},
    {"label": "ORG", "pattern": "Greenpeace"},
    {"label": "LOC", "pattern": "Durban"},
    {"label": "ORG", "pattern": "groundWork"},
    {"label": "ORG", "pattern": "GroundWork"},
    {"label": "ORG", "pattern": "Groundwork"},
    {"label": "ORG", "pattern": "groundwork"},
    {"label": "ORG", "pattern": "Earthlife Africa"},
    {"label": "ORG", "pattern": "Glencore"},
    {"label": "ORG", "pattern": "Optimum Coal"},
    {"label": "ORG", "pattern": "Optimum"},
    {"label": "LOC", "pattern": "Richards Bay Coal Terminal"},
    {"label": "ORG", "pattern": "Sasol"},
    {"label": "LOC", "pattern": "Komati Power Station"},
    {"label": "ORG", "pattern": "Exxaro Resources"},
    {"label": "ORG", "pattern": "Exxaro"},
    {"label": "ORG", "pattern": "Climate Justice Coalition"},
    {"label": "LOC", "pattern": "Kusile"},
    {"label": "LOC", "pattern": "Komati"},
    {"label": "LOC", "pattern": "KwaZulu-Natal"},
    {"label": "LOC", "pattern": "Vele"},
    {"label": "LOC", "pattern": "Vele coal mine"},
    {"label": "LOC", "pattern": "Vele mine"},
    {"label": "ORG", "pattern": "Ibutho Coal"},
    {"label": "LOC", "pattern": "Majuba Power Station"},
    {"label": "LOC", "pattern": "Majuba"},
    {"label": "LOC", "pattern": "Vunene"},
   # {"label": "ORG", "pattern": "Germany"},
   # {"label": "ORG", "pattern": "France"},
   # {"label": "UK", "pattern": "UK"},
   # {"label": "LOC", "pattern": "US"},


  ]
ruler.add_patterns(patterns)

# Function to predict entities using spaCy with custom entity ruler
def predict_ner_spacy(data):
    predictions = []
    for sentence, topic, year, month in tqdm(data, desc="Predicting NER with spaCy"):
        doc = nlp(sentence)
        entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'LOC']]
        predictions.append((sentence, topic, year, month, entities))
    return predictions

# Run prediction
predictions = predict_ner_spacy(data)

# Create an empty list to store the data for the DataFrame
df_data = []

for sentence, topic, year, month, entities in predictions:
    df_data.append([sentence, topic, year, month, entities])

# Create a DataFrame from the data list
df_ner = pd.DataFrame(df_data, columns=['Sentence', 'Dominant Topic', 'Year', 'Month', 'Entities'])

# Export the DataFrame as a CSV file
df_ner.to_csv('/content/drive/MyDrive/Colab Notebooks/ner_predictions_dtm_spacy5.csv', index=False)

# Display the DataFrame
print(df_ner.head(20))



# Initialize a list to store the expanded rows with correct entity types
expanded_data = []

# Iterate over the DataFrame rows
for index, row in df_ner.iterrows():
    sentence = row['Sentence']
    topic = row['Dominant Topic']
    year = row['Year']
    month = row['Month']
    entities = row['Entities']

    # Iterate over the entities to create separate rows for each actor
    for entity in entities:
        entity_name = entity[0]
        entity_type = entity[1]

        if entity_type == 'PERSON':
            expanded_data.append([sentence, topic, year, month, entity_name, None, None])
        elif entity_type == 'ORG':
            expanded_data.append([sentence, topic, year, month, None, entity_name, None])
        elif entity_type == 'LOC':
            expanded_data.append([sentence, topic, year, month, None, None, entity_name])

# Create a new DataFrame from the expanded data
df_expanded = pd.DataFrame(expanded_data, columns=['Sentence', 'Dominant Topic', 'Year', 'Month', 'PERSON', 'ORG', 'LOC'])

# Export the expanded DataFrame as a CSV file
df_expanded.to_csv('/content/drive/MyDrive/Colab Notebooks/ner_predictions_3_1.csv', index=False)

# Display the expanded DataFrame
print(df_expanded.head(20))

# Create a dictionary to store entities by topic
entities_by_topic = {topic: [] for topic in unique_topics if topic not in topics_of_no_interest}

for sentence, topic, year, month, entities in predictions:
    entities_by_topic[topic].extend(entities)

# Compute frequency of each entity per topic
summary_statistics = {}

for topic, entities in entities_by_topic.items():
    counter = Counter(entities)
    summary_statistics[topic] = counter.most_common()

# Create a DataFrame to display the summary statistics
summary_data = []

for topic, entity_list in summary_statistics.items():
    for (entity_text, entity_type), count in entity_list:
        summary_data.append([topic, entity_text, entity_type, count])

df_summary = pd.DataFrame(summary_data, columns=['Topic', 'Entity', 'Type', 'Count'])

# Export the DataFrame as a CSV file
df_summary.to_csv('/content/drive/MyDrive/Colab Notebooks/ner_spacy5_summary_statistics.csv', index=False)

# Display the DataFrame
print(df_summary.head(20))



Merged DataFrame:
   Unnamed: 0.1  Unnamed: 0  \
0             2         168   
1             3         261   
2             4         263   
3             5         360   
4             6         361   

                                        Jurisdiction  Location ContentType  \
0                                      International       NaN        News   
1                                      International       NaN        News   
2  U.S. Federal; International; Massachusetts; Wa...       NaN        News   
3                                      International       NaN        News   
4                                      International       NaN        News   

            Byline  WordLength  WebNewsUrl  Geography  NegativeNews  ...  \
0              NaN        1860         NaN        NaN           NaN  ...   
1     Ingi Salgado         700         NaN        NaN           NaN  ...   
2     Patrick Bond        1768         NaN        NaN           NaN  ...   
3  Eleanor Momberg    

Predicting NER with spaCy: 100%|██████████| 48767/48767 [12:34<00:00, 64.65it/s]


                                             Sentence  \
0   Women for Transformation, representing more th...   
1   Environmental groups rejected the dominance of...   
2   Many organisations, including the Southern Afr...   
3   Many organisations, including the Southern Afr...   
4   We\re confident this will prove more cost-effe...   
5   There was widespread support for Britton propo...   
6   Given the power relations in Congress, this is...   
7   They contrast the ineffectual pork-laden legis...   
8   In December, the EPA issued an endangerment fi...   
9   Hence halting the legislation in the Senate is...   
10  The major bills are now being pushed by John K...   
11  It is also based on cap and trade in which pol...   
12  The problem of relying upon these actors to sa...   
13  The boiler deals give the ruling party a multi...   
14  If Zuma aims, therefore, to persuade the world...   
15  The cuts promised by Sonjica just before Copen...   
16  In reality, cuts promised i

In [ ]:


# Load the CSV file with NER predictions
path_to_csv = '/content/drive/MyDrive/Colab Notebooks/ner_predictions_dtm_spacy5.csv'
df_ner = pd.read_csv(path_to_csv)

# Convert the string representation of lists in the Entities column back to lists
df_ner['Entities'] = df_ner['Entities'].apply(ast.literal_eval)

# Initialize a dictionary to store entities by topic
entities_by_topic = {topic: [] for topic in df_ner['Dominant Topic'].unique()}

# Iterate over the DataFrame rows to collect entities by topic
for index, row in df_ner.iterrows():
    entities = row['Entities']
    topic = row['Dominant Topic']
    # Normalize entities by removing backslashes and handling "The"/"the"
    normalized_entities = [(ent[0].replace('\\', '').replace('The ', '').replace('the ', '').replace("'s", ''), ent[1]) for ent in entities]
    entities_by_topic[topic].extend(normalized_entities)

# Initialize an empty list to store the summary DataFrames
summary_dfs = []

# Compute top entities for each topic
for topic, entities in entities_by_topic.items():
    # Count the frequency of each entity
    entity_counter = Counter(entities)
    # Get the top 10 entities
    top_entities = entity_counter.most_common(40)
    # Create summary DataFrame for the top entities
    df_summary = pd.DataFrame(top_entities, columns=['Entity', 'Count'])
    df_summary['Type'] = df_summary['Entity'].apply(lambda x: next((ent[1] for ent in entities if ent[0] == x), ''))
    df_summary['Topic'] = topic
    # Append the summary DataFrame to the list
    summary_dfs.append(df_summary)

# Concatenate all topic-specific summary DataFrames into a single DataFrame
df_combined_summary = pd.concat(summary_dfs, ignore_index=True)

# Export the combined summary DataFrame as a CSV file
df_combined_summary.to_csv('/content/drive/MyDrive/Colab Notebooks/top_entities_summary_by_topic5.csv', index=False)

# Display the combined summary DataFrame
print(df_combined_summary.head(40))


                                    Entity  Count Type         Topic
0                             (Eskom, ORG)    609       Eskom crisis
1                                (SA, ORG)     96       Eskom crisis
2                            (Kusile, LOC)     95       Eskom crisis
3                               (ANC, ORG)     56       Eskom crisis
4                             (Nersa, ORG)     48       Eskom crisis
5                        (Mpumalanga, LOC)     46       Eskom crisis
6                          (Treasury, ORG)     30       Eskom crisis
7                        (World Bank, ORG)     28       Eskom crisis
8                             (Sasol, ORG)     27       Eskom crisis
9                            (Durban, LOC)     22       Eskom crisis
10                     (Ramaphosa, PERSON)     20       Eskom crisis
11                           (Exxaro, ORG)     15       Eskom crisis
12                              (SAA, ORG)     14       Eskom crisis
13                         (Glenco

In [ ]:
# This is displaying a new row for each identified entity (so if >1 entity in same sentence, same sentence over >1 rows )

# Load the CSV file with NER predictions
path_to_csv = '/content/drive/MyDrive/Colab Notebooks/ner_predictions_dtm_spacy5.csv'
df_ner = pd.read_csv(path_to_csv)

# Convert the string representation of lists in the Entities column back to lists
df_ner['Entities'] = df_ner['Entities'].apply(ast.literal_eval)

# Initialize a list to store the expanded rows
expanded_data = []

# Iterate over the DataFrame rows
for index, row in df_ner.iterrows():
    sentence = row['Sentence']
    topic = row['Dominant Topic']
    year = row['Year']
    month = row['Month']
    entities = row['Entities']

    # Iterate over the entities to create separate rows for each actor
    for entity in entities:
        entity_name = entity[0]
        entity_type = entity[1]
        expanded_data.append([sentence, topic, year, month, entity_name, entity_type])

# Create a new DataFrame from the expanded data
df_expanded = pd.DataFrame(expanded_data, columns=['Sentence', 'Dominant Topic', 'Year', 'Month', 'Entity', 'Type'])

# Export the expanded DataFrame as a CSV file
df_expanded.to_csv('/content/drive/MyDrive/Colab Notebooks/ner_predictions_3_2.csv', index=False)

# Display the expanded DataFrame
print(df_expanded.head(20))


                                             Sentence  \
0   Women for Transformation, representing more th...   
1   Environmental groups rejected the dominance of...   
2   Many organisations, including the Southern Afr...   
3   Many organisations, including the Southern Afr...   
4   We\re confident this will prove more cost-effe...   
5   There was widespread support for Britton propo...   
6   Given the power relations in Congress, this is...   
7   They contrast the ineffectual pork-laden legis...   
8   They contrast the ineffectual pork-laden legis...   
9   They contrast the ineffectual pork-laden legis...   
10  In December, the EPA issued an endangerment fi...   
11  In December, the EPA issued an endangerment fi...   
12  Hence halting the legislation in the Senate is...   
13  The major bills are now being pushed by John K...   
14  The major bills are now being pushed by John K...   
15  The major bills are now being pushed by John K...   
16  The problem of relying upon

In [ ]:
import pandas as pd
import ast

# Lists of entities
societal_entities = [
    'Greenpeace', 'Earthlife Africa', 'groundWork', 'Greenpeace Africa', 'CER', 'Save Mapungubwe Coalition',
    'Endangered Wildlife Trust', 'Ezemvelo', 'Ted Blom', 'Chris Yelland', 'Prof Zeelie', 'Engelbrecht',
    'Von Blottnitz', 'Nelson Mandela Metropolitan University', 'CSIR', 'Just Coal', 'Vukani Environmental Justice Movement',
    'Mike Holland', 'Rico Euripidou', 'Centre for Environmental Rights', 'WWF', 'NGO', 'Birdlife SA', 'University of Cape Town',
    'Bench Marks Foundation', 'Federation for a Sustainable Environment' 'Global Energy Monitor'
]

unions_entities = [
    'Cosatu', 'NUM', 'National Union of Mineworkers', 'Numsa', 'Amcu', 'Association of Mineworkers and Construction Union',
    'Solidarity', 'Joseph Mathunjwa', 'Jim', 'Lesiba Seshoka', 'Congress of South African Trade Unions',
    'THE National Union of Mineworkers', 'Allied Workers Union'
]

other_business_entities = [
    'SAA', 'Transnet', 'Standard Bank', 'Nedbank', 'Gupta', 'Guptas', 'Brian Molefe', 'Tata', 'IPP', 'Google',
    'Shell', 'Mafutha', 'Royal Dutch Shell', 'Sunbird Energy', 'JSE'
]

mining_business_entities = [
    'Sasol', 'Exxaro', 'Glencore', 'Coal of Africa', 'CoAL', 'Ibutho Coal', 'John Wallington', 'Atha-Africa', 'Optimum',
    'BHP Billiton', 'BEE', 'BHP', 'Tegeta', 'ExxonMobil', 'MMTC', 'Minerals Council SA', 'Umsobomvu Coal',
    'Elitheni', 'AEMFC', 'Teke', 'SAEC', 'Mike Teke', 'Mazizi Msutu', 'Tegeta Exploration and Resources', 'Wescoal',
    'Umthombo Resources', 'ICVL', 'Chamber of Mines', 'Shanduka Coal', 'Delmas Coal', 'Kangra Coal', 'Impala Platinum',
    'AngloGold Ashanti', 'Firestone', 'Xstrata', 'Continental Coal', 'Xstrata Coal'
]

international_actors = [
    'World Bank', 'G20', 'EU', 'UN', 'Intergovernmental Panel on Climate Change (IPCC', 'Xi Jinping', 'Xi',
    'International Energy Agency', 'World Health Organisation', 'the Conference of the Parties', 'United Nations', 'the European Union',
    'IPG', 'International Partners Group', 'Biden', ' Joe Biden', 'Brics'
]

political_entities = [
    'ANC', 'Nersa', 'Treasury', 'Ramaphosa', 'Cyril Ramaphosa', 'Mboweni', 'National Treasury', 'Gwede Mantashe',
    'Gordhan', 'Pravin Gordhan', 'National Energy Regulator of SA', 'Barbara Creecy', 'PCC', 'Presidential Climate Commission',
    'Department of Environmental Affairs', 'Department of Mineral Resources', 'Department of Water Affairs', 'DEA',
    'Edna Molewa', 'Malusi Gigaba', 'Zuma', 'EFF', 'Jacob Zuma', 'Malema', 'Parliament', 'Parks Tau', 'Kelvin',
    'Department of Trade and Industry', 'Department of Energy', 'Fikile Mbalula', 'Department of Mineral Resources and Energy',
    'DMRE', 'Susan Shabangu', 'Ms Shabangu', 'NPA', 'CCMA', 'Commission for Conciliation, Mediation and Arbitration (CCMA',
    'Democratic Alliance', 'DA', 'President Ramaphosa', "Jeff Radebe" # These 4 added by myself
]

eskom_entities = [
    'Eskom', 'De Ruyter', 'Andrew Etzinger', 'Brian Dames', 'Joffe', 'Hilary Joffe', 'Tshediso Matona', 'Brian Molefe',
    'Koko', 'Matshela Koko'
]

# Load the CSV file with NER predictions
path_to_csv = '/content/drive/MyDrive/Colab Notebooks/ner_predictions_dtm_spacy5.csv'
df_ner = pd.read_csv(path_to_csv)

# Convert the string representation of lists in the Entities column back to lists
df_ner['Entities'] = df_ner['Entities'].apply(ast.literal_eval)

# Initialize a list to store the expanded rows
expanded_data = []

# Iterate over the DataFrame rows
for index, row in df_ner.iterrows():
    sentence = row['Sentence']
    topic = row['Dominant Topic']
    year = row['Year']
    month = row['Month']
    entities = row['Entities']

    # Iterate over the entities to create separate rows for each actor
    for entity in entities:
        entity_name = entity[0]
        entity_type = entity[1]

        # Determine the category of the entity
        if entity_name in societal_entities:
            category = 'Societal'
        elif entity_name in unions_entities:
            category = 'Unions'
        elif entity_name in other_business_entities:
            category = 'Other Business'
        elif entity_name in mining_business_entities:
            category = 'Mining Business'
        elif entity_name in international_actors:
            category = 'International'
        elif entity_name in political_entities:
            category = 'Political'
        elif entity_name in eskom_entities:
            category = 'Eskom'
        else:
            category = 'No Entity'

        expanded_data.append([sentence, topic, year, month, entity_name, entity_type, category])

# Create a new DataFrame from the expanded data
df_expanded = pd.DataFrame(expanded_data, columns=['Sentence', 'Dominant Topic', 'Year', 'Month', 'Entity', 'Type', 'Category'])

# Export the expanded DataFrame as a CSV file
df_expanded.to_csv('/content/drive/MyDrive/Colab Notebooks/ner_predictions_with_categories.csv', index=False)

# Display the expanded DataFrame
print(df_expanded.head(20))


                                             Sentence  \
0   Women for Transformation, representing more th...   
1   Environmental groups rejected the dominance of...   
2   Many organisations, including the Southern Afr...   
3   Many organisations, including the Southern Afr...   
4   We\re confident this will prove more cost-effe...   
5   There was widespread support for Britton propo...   
6   Given the power relations in Congress, this is...   
7   They contrast the ineffectual pork-laden legis...   
8   They contrast the ineffectual pork-laden legis...   
9   They contrast the ineffectual pork-laden legis...   
10  In December, the EPA issued an endangerment fi...   
11  In December, the EPA issued an endangerment fi...   
12  Hence halting the legislation in the Senate is...   
13  The major bills are now being pushed by John K...   
14  The major bills are now being pushed by John K...   
15  The major bills are now being pushed by John K...   
16  The problem of relying upon

In [ ]:
# Summary: How many observations per category
category_counts = df_expanded['Category'].value_counts()
print(category_counts)

# Summary: How many observations per category per topic
category_topic_counts = df_expanded.groupby(['Category', 'Dominant Topic']).size().reset_index(name='Counts')
print(category_topic_counts)
# Export the summaries to CSV files
category_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_counts.csv', index=True)
category_topic_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_topic_counts.csv', index=False)

Category
No Entity          39883
Mining Business     6572
Eskom               5573
Political           2481
Other Business      1580
Societal             710
Unions               533
International        383
Name: count, dtype: int64
    Category                         Dominant Topic  Counts
0      Eskom                          Air pollution     108
1      Eskom                   Anti-mining activism      72
2      Eskom                  Climate change impact      58
3      Eskom                      Coal labor issues      55
4      Eskom                   Coal trade relations     102
..       ...                                    ...     ...
108   Unions                     Mining investments      10
109   Unions            Safety & illegal activities      11
110   Unions             State capture & corruption      29
111   Unions           State intervention in mining      17
112   Unions  Transition politics & climate finance      21

[113 rows x 3 columns]


In [ ]:
import pandas as pd
import ast

# Lists of entities
societal_entities = [
    'Greenpeace', 'Earthlife Africa', 'groundWork', 'Greenpeace Africa', 'CER', 'Save Mapungubwe Coalition',
    'Endangered Wildlife Trust', 'Ezemvelo', 'Ted Blom', 'Chris Yelland', 'Prof Zeelie', 'Engelbrecht',
    'Von Blottnitz', 'Nelson Mandela Metropolitan University', 'CSIR', 'Just Coal', 'Vukani Environmental Justice Movement',
    'Mike Holland', 'Rico Euripidou'
]

unions_entities = [
    'Cosatu', 'NUM', 'National Union of Mineworkers', 'Numsa', 'Amcu', 'Association of Mineworkers and Construction Union',
    'Solidarity', 'Joseph Mathunjwa', 'Jim', 'Lesiba Seshoka', 'Congress of South African Trade Unions',
    'THE National Union of Mineworkers', 'Allied Workers Union'
]

other_business_entities = [
    'SAA', 'Transnet', 'Standard Bank', 'Nedbank', 'Gupta', 'Guptas', 'Brian Molefe', 'Tata', 'IPP', 'Google',
    'Shell', 'Mafutha', 'Royal Dutch Shell', 'Sunbird Energy', 'JSE'
]

mining_business_entities = [
    'Sasol', 'Exxaro', 'Glencore', 'Coal of Africa', 'CoAL', 'Ibutho Coal', 'John Wallington', 'Atha-Africa', 'Optimum',
    'BHP Billiton', 'BEE', 'BHP', 'Tegeta', 'ExxonMobil', 'MMTC', 'Minerals Council SA', 'Umsobomvu Coal',
    'Elitheni', 'AEMFC', 'Teke', 'SAEC', 'Mike Teke', 'Mazizi Msutu', 'Tegeta Exploration and Resources', 'Wescoal',
    'Umthombo Resources', 'ICVL', 'Chamber of Mines', 'Shanduka Coal', 'Delmas Coal', 'Kangra Coal', 'Impala Platinum',
    'AngloGold Ashanti', 'Firestone', 'Xstrata', 'Continental Coal', 'Xstrata Coal'
]

international_actors = [
    'World Bank', 'G20', 'EU', 'UN', 'Intergovernmental Panel on Climate Change (IPCC', 'Xi Jinping',
    'International Energy Agency', 'World Health Organisation'
]

political_entities = [
    'ANC', 'Nersa', 'Treasury', 'Ramaphosa', 'Cyril Ramaphosa', 'Mboweni', 'National Treasury', 'Gwede Mantashe',
    'Gordhan', 'Pravin Gordhan', 'National Energy Regulator of SA', 'Barbara Creecy', 'PCC', 'Presidential Climate Commission',
    'Department of Environmental Affairs', 'Department of Mineral Resources', 'Department of Water Affairs', 'DEA',
    'Edna Molewa', 'Malusi Gigaba', 'Zuma', 'EFF', 'Jacob Zuma', 'Malema', 'Parliament', 'Parks Tau', 'Kelvin',
    'Department of Trade and Industry', 'Department of Energy', 'Fikile Mbalula', 'Department of Mineral Resources and Energy',
    'DMRE', 'Susan Shabangu', 'Ms Shabangu', 'NPA', 'CCMA', 'Commission for Conciliation, Mediation and Arbitration (CCMA'
]



eskom_entities = [
    'Eskom', 'De Ruyter', 'Andrew Etzinger', 'Brian Dames', 'Joffe', 'Hilary Joffe', 'Tshediso Matona', 'Brian Molefe',
    'Koko', 'Matshela Koko'
]

# Topic categories
State_mining  = [
    'State capture & corruption', 'State intervention in mining', 'Daily political business', 'Coal trade relations'
]

Mining_industry = [
    'Mining investments'
]

Mining_communities = [
    'Safety & illegal activities', 'Air pollution', 'Anti-mining activism', 'Coal labor issues'
]

Transition = [
    'Transition politics & climate finance', 'Energy policies & renewables', 'Climate change impact'
]

Energy_crisis = [
    'Eskom crisis', 'Oil & gas alternatives', 'Electricity crisis'
]

# Load the CSV file with NER predictions
path_to_csv = '/content/drive/MyDrive/Colab Notebooks/ner_predictions_dtm_spacy5.csv'
df_ner = pd.read_csv(path_to_csv)

# Convert the string representation of lists in the Entities column back to lists
df_ner['Entities'] = df_ner['Entities'].apply(ast.literal_eval)

# Initialize a list to store the expanded rows
expanded_data = []

# Iterate over the DataFrame rows
for index, row in df_ner.iterrows():
    sentence = row['Sentence']
    topic = row['Dominant Topic']
    year = row['Year']
    month = row['Month']
    entities = row['Entities']

    # Determine the topic category
    if topic in State_mining:
        topic_category = 'State role in mining'
    elif topic in Mining_industry:
        topic_category = 'Mining industry'
    elif topic in Mining_communities:
        topic_category = 'Mining communities'
    elif topic in Transition:
        topic_category = 'Transition Politics'
    else topic in Energy_crisis:
        topic_category = 'Energy crisis'
    #else:
    #    topic_category = 'None'

    # Iterate over the entities to create separate rows for each actor
    for entity in entities:
        entity_name = entity[0]
        entity_type = entity[1]

        # Determine the category of the entity
        if entity_name in societal_entities:
            category = 'Societal'
        elif entity_name in unions_entities:
            category = 'Unions'
        elif entity_name in other_business_entities:
            category = 'Other Business'
        elif entity_name in mining_business_entities:
            category = 'Mining Business'
        elif entity_name in international_actors:
            category = 'International'
        elif entity_name in political_entities:
            category = 'Political'
        elif entity_name in eskom_entities:
            category = 'Eskom'
        else:
            category = 'No Entity'

        expanded_data.append([sentence, topic, year, month, entity_name, entity_type, category, topic_category])

# Create a new DataFrame from the expanded data
df_expanded = pd.DataFrame(expanded_data, columns=['Sentence', 'Dominant Topic', 'Year', 'Month', 'Entity', 'Type', 'Category', 'Topic Category'])

# Export the expanded DataFrame as a CSV file
df_expanded.to_csv('/content/drive/MyDrive/Colab Notebooks/ner_predictions_with_categories.csv', index=False)

# Display the expanded DataFrame
print(df_expanded.head(20))

# Summary: How many observations per category
category_counts = df_expanded['Category'].value_counts()
print(category_counts)

# Summary: How many observations per category per topic
category_topic_counts = df_expanded.groupby(['Category', 'Dominant Topic']).size().reset_index(name='Counts')
print(category_topic_counts)

# Summary: How many observations per topic category
topic_category_counts = df_expanded['Topic Category'].value_counts()
print(topic_category_counts)

# Summary: How many observations per category per topic category
category_topic_category_counts = df_expanded.groupby(['Category', 'Topic Category']).size().reset_index(name='Counts')
print(category_topic_category_counts)

# Export the summaries to CSV files
category_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_counts.csv', index=True)
category_topic_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_topic_counts.csv', index=False)
topic_category_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/topic_category_counts.csv', index=True)
category_topic_category_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_topic_category_counts.csv', index=False)


                                             Sentence  \
0   Women for Transformation, representing more th...   
1   Environmental groups rejected the dominance of...   
2   Many organisations, including the Southern Afr...   
3   Many organisations, including the Southern Afr...   
4   We\re confident this will prove more cost-effe...   
5   There was widespread support for Britton propo...   
6   Given the power relations in Congress, this is...   
7   They contrast the ineffectual pork-laden legis...   
8   They contrast the ineffectual pork-laden legis...   
9   They contrast the ineffectual pork-laden legis...   
10  In December, the EPA issued an endangerment fi...   
11  In December, the EPA issued an endangerment fi...   
12  Hence halting the legislation in the Senate is...   
13  The major bills are now being pushed by John K...   
14  The major bills are now being pushed by John K...   
15  The major bills are now being pushed by John K...   
16  The problem of relying upon

In [ ]:
import pandas as pd
import ast

# Lists of entities
societal_entities = [
    'Greenpeace', 'Earthlife Africa', 'groundWork', 'Greenpeace Africa', 'CER', 'Save Mapungubwe Coalition',
    'Endangered Wildlife Trust', 'Ezemvelo', 'Ted Blom', 'Chris Yelland', 'Prof Zeelie', 'Engelbrecht',
    'Von Blottnitz', 'Nelson Mandela Metropolitan University', 'CSIR', 'Just Coal', 'Vukani Environmental Justice Movement',
    'Mike Holland', 'Rico Euripidou'
]

unions_entities = [
    'Cosatu', 'NUM', 'National Union of Mineworkers', 'Numsa', 'Amcu', 'Association of Mineworkers and Construction Union',
    'Solidarity', 'Joseph Mathunjwa', 'Jim', 'Lesiba Seshoka', 'Congress of South African Trade Unions',
    'THE National Union of Mineworkers', 'Allied Workers Union'
]

other_business_entities = [
    'SAA', 'Transnet', 'Standard Bank', 'Nedbank', 'Gupta', 'Guptas', 'Brian Molefe', 'Tata', 'IPP', 'Google',
    'Shell', 'Mafutha', 'Royal Dutch Shell', 'Sunbird Energy', 'JSE'
]

mining_business_entities = [
    'Sasol', 'Exxaro', 'Glencore', 'Coal of Africa', 'CoAL', 'Ibutho Coal', 'John Wallington', 'Atha-Africa', 'Optimum',
    'BHP Billiton', 'BEE', 'BHP', 'Tegeta', 'ExxonMobil', 'MMTC', 'Minerals Council SA', 'Umsobomvu Coal',
    'Elitheni', 'AEMFC', 'Teke', 'SAEC', 'Mike Teke', 'Mazizi Msutu', 'Tegeta Exploration and Resources', 'Wescoal',
    'Umthombo Resources', 'ICVL', 'Chamber of Mines', 'Shanduka Coal', 'Delmas Coal', 'Kangra Coal', 'Impala Platinum',
    'AngloGold Ashanti', 'Firestone', 'Xstrata', 'Continental Coal', 'Xstrata Coal'
]

international_actors = [
    'World Bank', 'G20', 'EU', 'UN', 'Intergovernmental Panel on Climate Change (IPCC', 'Xi Jinping',
    'International Energy Agency', 'World Health Organisation'
]

political_entities = [
    'ANC', 'Nersa', 'Treasury', 'Ramaphosa', 'Cyril Ramaphosa', 'Mboweni', 'National Treasury', 'Gwede Mantashe',
    'Gordhan', 'Pravin Gordhan', 'National Energy Regulator of SA', 'Barbara Creecy', 'PCC', 'Presidential Climate Commission',
    'Department of Environmental Affairs', 'Department of Mineral Resources', 'Department of Water Affairs', 'DEA',
    'Edna Molewa', 'Malusi Gigaba', 'Zuma', 'EFF', 'Jacob Zuma', 'Malema', 'Parliament', 'Parks Tau', 'Kelvin',
    'Department of Trade and Industry', 'Department of Energy', 'Fikile Mbalula', 'Department of Mineral Resources and Energy',
    'DMRE', 'Susan Shabangu', 'Ms Shabangu', 'NPA', 'CCMA', 'Commission for Conciliation, Mediation and Arbitration (CCMA'
]

eskom_entities = [
    'Eskom', 'De Ruyter', 'Andrew Etzinger', 'Brian Dames', 'Joffe', 'Hilary Joffe', 'Tshediso Matona', 'Brian Molefe',
    'Koko', 'Matshela Koko'
]

# Topic categories
State_mining  = [
    'State capture & corruption', 'State intervention in mining', 'Daily political business', 'Coal trade relations'
]

Mining_industry = [
    'Mining investments'
]

Mining_communities = [
    'Safety & illegal activities', 'Air pollution', 'Anti-mining activism', 'Coal labor issues'
]

Transition = [
    'Transition politics & climate finance', 'Energy policies & renewables', 'Climate change impact'
]

Energy_crisis = [
    'Eskom crisis', 'Oil & gas alternatives', 'Electricity crisis'
]

# Entity normalization mapping
entity_mapping = {
    'Greenpeace Africa': 'Greenpeace',
    'Zuma': 'Jacob Zuma',
    'BHP': 'BHP Billiton',
    'Ramaphosa': 'Cyril Ramaphosa',
    'Guptas': 'Gupta',
    'National Union of Mineworkers': 'NUM',
    'THE National Union of Mineworkers': 'NUM',
    'Gwede Mantashe': 'Gwede Mantashe',
    'Mantashe': 'Gwede Mantashe'
}

# Load the CSV file with NER predictions
path_to_csv = '/content/drive/MyDrive/Colab Notebooks/ner_predictions_dtm_spacy5.csv'
df_ner = pd.read_csv(path_to_csv)

# Convert the string representation of lists in the Entities column back to lists
df_ner['Entities'] = df_ner['Entities'].apply(ast.literal_eval)

# Initialize a list to store the expanded rows
expanded_data = []

# Iterate over the DataFrame rows
for index, row in df_ner.iterrows():
    sentence = row['Sentence']
    topic = row['Dominant Topic']
    year = row['Year']
    month = row['Month']
    entities = row['Entities']

    # Determine the topic category
    if topic in State_mining:
        topic_category = 'State role in mining'
    elif topic in Mining_industry:
        topic_category = 'Mining industry'
    elif topic in Mining_communities:
        topic_category = 'Mining communities'
    elif topic in Transition:
        topic_category = 'Transition Politics'
    else:
        topic_category = 'Energy crisis'

    # Iterate over the entities to create separate rows for each actor
    for entity in entities:
        entity_name = entity[0]
        entity_type = entity[1]

        # Normalize the entity name
        if entity_name in entity_mapping:
            entity_name = entity_mapping[entity_name]

        # Determine the category and broad category of the entity
        if entity_name in societal_entities:
            category = 'Societal'
            broad_category = 'Societal'
        elif entity_name in unions_entities:
            category = 'Unions'
            broad_category = 'Societal'
        elif entity_name in other_business_entities:
            category = 'Other Business'
            broad_category = 'Business'
        elif entity_name in mining_business_entities:
            category = 'Mining Business'
            broad_category = 'Business'
        elif entity_name in international_actors:
            category = 'International'
            broad_category = 'Political'
        elif entity_name in political_entities:
            category = 'Political'
            broad_category = 'Political'
        elif entity_name in eskom_entities:
            category = 'Eskom'
            broad_category = 'Eskom'
        else:
            category = 'No Entity'
            broad_category = 'No Entity'

        expanded_data.append([sentence, topic, year, month, entity_name, entity_type, category, topic_category, broad_category])

# Create a new DataFrame from the expanded data
df_expanded = pd.DataFrame(expanded_data, columns=['Sentence', 'Dominant Topic', 'Year', 'Month', 'Entity', 'Type', 'Category', 'Topic Category', 'Broad Category'])
df_expanded = df_expanded.drop_duplicates(subset=['Sentence', 'Entity', 'Year', 'Month', 'Dominant Topic'])

# Export the expanded DataFrame as a CSV file
df_expanded.to_csv('/content/drive/MyDrive/Colab Notebooks/ner_predictions_with_categories3.csv', index=False)

# Display the expanded DataFrame
print(df_expanded.head(20))

# Summary: How many observations per category
category_counts = df_expanded['Category'].value_counts()
print(category_counts)

# Summary: How many observations per category per topic
category_topic_counts = df_expanded.groupby(['Category', 'Dominant Topic']).size().reset_index(name='Counts')
print(category_topic_counts)

# Summary: How many observations per topic category
topic_category_counts = df_expanded['Topic Category'].value_counts()
print(topic_category_counts)

# Summary: How many observations per category per topic category
category_topic_category_counts = df_expanded.groupby(['Category', 'Topic Category']).size().reset_index(name='Counts')
print(category_topic_category_counts)

# Summary: How many observations per broad category
broad_category_counts = df_expanded['Broad Category'].value_counts()
print(broad_category_counts)

# Summary: How many observations per broad category per topic category
broad_category_topic_category_counts = df_expanded.groupby(['Broad Category', 'Topic Category']).size().reset_index(name='Counts')
print(broad_category_topic_category_counts)

# Summary: How many observations per broad category per topic category
category_entity_counts = df_expanded.groupby(['Category', 'Entity']).size().reset_index(name='Counts')
print(category_entity_counts)

# Export the summaries to CSV files
#category_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_counts.csv', index=True)
#category_topic_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_topic_counts.csv', index=False)
#topic_category_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/topic_category_counts.csv', index=True)
#category_topic_category_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_topic_category_counts.csv', index=False)
#category_entity_counts.to_csv('/content/drive/MyDrive/Colab Notebooks/category_entity_counts.csv', index=False)


                                             Sentence  \
0   Women for Transformation, representing more th...   
1   Environmental groups rejected the dominance of...   
2   Many organisations, including the Southern Afr...   
4   We\re confident this will prove more cost-effe...   
5   There was widespread support for Britton propo...   
6   Given the power relations in Congress, this is...   
7   They contrast the ineffectual pork-laden legis...   
8   They contrast the ineffectual pork-laden legis...   
9   They contrast the ineffectual pork-laden legis...   
10  In December, the EPA issued an endangerment fi...   
11  In December, the EPA issued an endangerment fi...   
12  Hence halting the legislation in the Senate is...   
13  The major bills are now being pushed by John K...   
14  The major bills are now being pushed by John K...   
15  The major bills are now being pushed by John K...   
16  The problem of relying upon these actors to sa...   
17  The problem of relying upon

In [ ]:
df_subset = df_expanded[df_expanded['Category'] != 'No Entity']

df_subset.to_csv('/content/drive/MyDrive/Colab Notebooks/filtered_ner_predictions_with_categories3.csv', index=False)

### Now make Rule looking at the top 40 entries -> Create a dataset containing only sentences connected to one of the top 40 actors (excluding the irrelevant/ false ones, e.g. "coal" or "irp". What about location?

Take out:
"SAA", "Integrated Resource Plan",
"Coal", "MW", "Copenhagen", "first onMail & Guardian", "SA", "House", "Mozambique", "LNG",


In [ ]:

# Load the CSV file with top 40 entities
path_to_top_entities = '/content/drive/MyDrive/Colab Notebooks/top_40_entities_summary.csv'
df_top_entities = pd.read_csv(path_to_top_entities)

# Filter out irrelevant terms
irrelevant_terms = ["SAA", "Integrated Resource Plan", "Coal", "MW", "Copenhagen",
                    "first onMail & Guardian", "SA", "House", "Mozambique", "LNG"]
filtered_entities = df_top_entities[~df_top_entities['Entity'].isin(irrelevant_terms)]['Entity'].tolist()

# Load the CSV file with NER predictions
path_to_ner_predictions = '/content/drive/MyDrive/Colab Notebooks/ner_predictions_dtm_spacy5.csv'
df_ner = pd.read_csv(path_to_ner_predictions)

# Convert the string representation of lists in the Entities column back to lists
df_ner['Entities'] = df_ner['Entities'].apply(ast.literal_eval)

# Initialize an empty list to store relevant rows
relevant_rows = []

# Iterate over the DataFrame rows to filter and expand rows
for index, row in df_ner.iterrows():
    entities = row['Entities']
    sentence = row['Sentence']
    topic = row['Dominant Topic']
    year = row['Year']
    month = row['Month']

    # Check if any entity matches the filtered top entities
    for entity, entity_type in entities:
        if entity in filtered_entities and entity_type != 'LOC':  # Exclude locations
            relevant_rows.append([sentence, entity, topic, year, month])

# Create a DataFrame from the relevant rows
df_relevant_sentences = pd.DataFrame(relevant_rows, columns=['Sentence', 'Entity', 'Dominant Topic', 'Year', 'Month'])

# Export the filtered dataset as a CSV file
df_relevant_sentences.to_csv('/content/drive/MyDrive/Colab Notebooks/relevant_sentences_expanded.csv', index=False)

# Display the first few rows of the filtered dataset
print(df_relevant_sentences.head(20))


                                             Sentence Entity  \
0   Environmental groups rejected the dominance of...  Eskom   
1   The problem of relying upon these actors to sa...  Eskom   
2   The problem of relying upon these actors to sa...    ANC   
3   In reality, cuts promised in the Long-Term Mit...  Sasol   
4   As the Eskom tariff hearings organised by the ...  Eskom   
5   As the Eskom tariff hearings organised by the ...  Eskom   
6   In reality, cuts promised in the Long-Term Mit...  Sasol   
7   As the Eskom tariff hearings organised by the ...  Eskom   
8   As the Eskom tariff hearings organised by the ...  Eskom   
9   Yet the purpose of the new build programme is ...  Eskom   
10  The department called this month for public co...  Eskom   
11  The department called this month for public co...  Eskom   
12  Several mines are expected to be developed in ...  Eskom   
13  Several mines are expected to be developed in ...  Eskom   
14  Several mines are expected to be dev

In [ ]:
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
